<a href="https://colab.research.google.com/github/malowana/NLP_budzet_obywatelski/blob/main/BO_analiza_morfologiczna2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!pip install -U spacy

In [9]:
!pip install morfeusz2

In [21]:
# instalacja morfeusza
!wget -O - http://download.sgjp.pl/apt/sgjp.gpg.key|sudo apt-key add -
!sudo apt-add-repository http://download.sgjp.pl/apt/ubuntu
!sudo apt update
!sudo apt install morfeusz2
!sudo apt install python3-morfeusz2


# instalacja modelu IPI PAN dla języka polskiego
!wget "http://zil.ipipan.waw.pl/SpacyPL?action=AttachFile&do=get&target=pl_spacy_model_morfeusz-0.1.3.tar.gz"
!mv 'SpacyPL?action=AttachFile&do=get&target=pl_spacy_model_morfeusz-0.1.3.tar.gz' pl_spacy_model_morfeusz-0.1.3.tar.gz
!python3 -m pip install pl_spacy_model_morfeusz-0.1.3.tar.gz

# linkowanie modelu do spaCy
!python3 -m spacy link pl_spacy_model_morfeusz pl_spacy_model_morfeusz -f

--2022-01-16 21:44:37--  http://download.sgjp.pl/apt/sgjp.gpg.key
Resolving download.sgjp.pl (download.sgjp.pl)... 193.0.67.154
Connecting to download.sgjp.pl (download.sgjp.pl)|193.0.67.154|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1029 (1.0K) [application/pgp-keys]
Saving to: ‘STDOUT’

-                   100%[===================>]   1.00K  --.-KB/s    in 0s      

2022-01-16 21:44:38 (234 MB/s) - written to stdout [1029/1029]

OK
Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic In

In [ ]:
#spacy.load('pl_spacy_model_morfeusz')

In [22]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from gensim.utils import simple_preprocess
from collections import Counter
import gc
import morfeusz2
import spacy
from spacy.lang.pl.stop_words import STOP_WORDS

In [24]:
morf = morfeusz2.Morfeusz()

In [26]:
for analysis in morf.analyse("sadzimy rośliny zimozielone budowa dużego skateparkuu na gocławiu dla całej pragi-południe jako wizytówka prawej strony wisły domki dla jeży nad balatonem nie chcesz mieć szczurów"):
  # na wyjściu z morf.analyse dostajemy listę interpretacji, niektóre tokeny mogą mieć więcej niż jedną interpretację, jak tu z tokenem "życia"
  start_ind, end_ind, interp = analysis
  form, lemma, tag, qualifiers, _ = interp
  try:
    pos, feats = tag.split(":", 1)
  except ValueError: # podana forma nie ma cech morfosyntaktycznych
    pos, feats = tag, ""
  print("{0:4} {1:4} {2:8} {3:8} {4:6} {5:20} {6:15}".format(start_ind, end_ind, form, lemma, pos, feats, str(qualifiers)))

   0    1 sadzimy  sadzić   fin    pl:pri:imperf        []             
   1    2 rośliny  roślina  subst  sg:gen:f             ['nazwa_pospolita']
   1    2 rośliny  roślina  subst  pl:nom.acc.voc:f     ['nazwa_pospolita']
   2    3 zimozielone zimozielony adj    pl:acc:m2.m3.f.n:pos []             
   2    3 zimozielone zimozielony adj    pl:nom.voc:m2.m3.f.n:pos []             
   2    3 zimozielone zimozielony adj    sg:acc:n:pos         []             
   2    3 zimozielone zimozielony adj    sg:nom.voc:n:pos     []             
   3    4 budowa   budowa   subst  sg:nom:f             ['nazwa_pospolita']
   4    5 dużego   duży     adj    sg:acc:m1.m2:pos     []             
   4    5 dużego   duży     adj    sg:gen:m1.m2.m3.n:pos []             
   5    6 skateparkuu skateparkuu ign                         []             
   6    7 na       na:I     interj                      []             
   6    7 na       na:P     prep   acc                  []             
   6    7 na     